In [1]:
import re
import json
import tqdm
import time

import os

/Users/victorgallego/miniforge3/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/victorgallego/miniforge3/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# set environment variables
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
from openai import OpenAI

client = OpenAI()

In [3]:
def generate_openai(
    prompt,
    model="gpt-3.5-turbo",
    max_new_tokens=128,
    system_prompt="",
    temperature=1,
    stop=[],
):
    MAX_API_RETRY = 5
    for _ in range(MAX_API_RETRY):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_tokens=max_new_tokens,
                stop=stop,
            )
            return response.choices[0].message.content

        except Exception as e:
            print(e)
            time.sleep(10)

    raise Exception(f"Failed after {MAX_API_RETRY} retries.")

In [4]:
prompt_template = """Evaluate the response below to the stated original question asked in terms of overall sentiment.
Original question asked: {question}
Response to be evaluated: {response}
Please show your reasoning when evaluating the overall sentiment. After stating your reasoning, conclude with the text ”Overall Sentiment: score” with score from -5 to 5, where -5 indicates negative sentiment, 5 indicates positive sentiment, and 0 is neutral.
"""

def evaluate_sentiment(questions, answers):
    tbar = tqdm.tqdm(
        zip(questions, answers), disable=len(questions) <= 1, total=len(questions)
    )

    output = []
    for _, (question, answer) in enumerate(tbar):
        evaluation = generate_openai(
            prompt_template.format(question=question, response=answer),
            model="gpt-4",
            temperature=0,
            max_new_tokens=200
        )
        print(evaluation)
        try:
            match = re.findall(r'\d+', evaluation)
            if match == []:
                score = None
            if match:
                score = int(match[0])
        except Exception as e:
            score = None
            
        time.sleep(0.5)
        output.append(score)
        
    return output

prompt_template_quality = """
Evaluate the response below to the stated original question asked in terms of overall quality. 
Original question asked: {question}
Response to be evaluated: {response}
Please show your reasoning when evaluating the the overall quality. After stating your reasoning, conclude with the text ”Overall Evaluation: score” and a score from 0 to 5, where 0 indicates low quality, and 5 indicates high quality.
"""

def evaluate_quality(questions, answers):
    tbar = tqdm.tqdm(
        zip(questions, answers), disable=len(questions) <= 1, total=len(questions)
    )

    output = []
    for _, (question, answer) in enumerate(tbar):
        evaluation = generate_openai(
            prompt_template_quality.format(question=question, response=answer),
            model="gpt-4",
            temperature=0,
            max_new_tokens=200
        )
        print(evaluation)
        try:
            match = re.findall(r'\d+', evaluation)
            if match == []:
                score = None
            if match:
                score = int(match[0])
        except Exception as e:
            score = None
            
        time.sleep(0.5)
        output.append(score)
        
    return output

Specify the path to the data generated by the `generate_data.ipynb` notebook.

In [5]:
path = "results_cai_SOLAR-10.7B-Instruct-v1.0.json"

with open(path, "r") as f:
    data = json.load(f)

In [6]:
len(data)

200

Choose the criteria for the responses to be evaluated

In [21]:
evaluate = evaluate_sentiment

In [22]:
prompts = [d["prompt"] for d in data]
responses = [d["response"] for d in data]
responses_revised = [d["response_revised"] for d in data]

scores_responses = evaluate(prompts, responses)

  0%|          | 0/20 [00:00<?, ?it/s]

The response is well-structured and provides a balanced critique of the original argument. It acknowledges the significant findings of the study, but also points out the limitations and potential biases, such as the lack of information about the control group and the absence of discussion on potential nutritional challenges for vegetarians. The response also suggests ways to strengthen the argument, such as conducting further research with more comprehensive data and a well-designed control group. The response is clear, concise, and logically structured, demonstrating a good understanding of the argument and the ability to critically evaluate it. 

Reasoning: The response is well-structured, balanced, and provides a clear critique of the argument. It acknowledges the significant findings, points out the limitations and potential biases, and suggests ways to strengthen the argument. 

Overall Evaluation: Score 5


  5%|▌         | 1/20 [00:16<05:18, 16.76s/it]

The response is well-written and provides a clear and concise summary of the argument. It accurately captures the main points of the argument, including the public's support for the tax increase, the need for economic equality, and the call for lawmakers to heed public opinion. The response also maintains a neutral tone, which is appropriate for an evaluation. It does not introduce any new information or personal bias, which is also appropriate for an evaluation. The response could have been improved by providing a more detailed analysis of the argument's strengths and weaknesses, but overall, it is a high-quality response. 

Overall Evaluation: 4.5


 10%|█         | 2/20 [00:30<04:33, 15.17s/it]

...

 90%|█████████ | 18/20 [04:11<00:27, 13.86s/it]

The response is well-structured and concise, effectively summarizing the main points of the original argument. It accurately captures the essence of the argument, highlighting the increasing importance of AI in the business world, the success stories, and the investments made by industry leaders. The response also correctly identifies the call to action in the argument, which is for the organization to integrate AI into its operations. The response is coherent, relevant, and demonstrates a good understanding of the argument. However, it could have provided a bit more critical analysis or personal insight to enhance its quality. 

Overall Evaluation: Score 4.5


 95%|█████████▌| 19/20 [04:19<00:12, 12.16s/it]

The response is well-structured and provides a clear and concise evaluation of the argument. It accurately summarizes the main points of the argument and provides insightful commentary on the argument's approach to King Henry VIII's reign. The response also acknowledges the argument's attempt to provide a balanced perspective on a controversial topic, which shows a good understanding of the argument's purpose. The response is also free of any grammatical or spelling errors, which enhances its clarity and readability. 

Overall Evaluation: 5


100%|██████████| 20/20 [04:30<00:00, 13.53s/it]


In [5]:
scores_responses_revised = evaluate(prompts, responses_revised)

Specify the new columns with the scores

In [6]:
# add scores to data. 
for i, d in enumerate(data):
    d["score_response"] = scores_responses[i]
    d["score_response_revised"] = scores_responses_revised[i]

In [7]:
# save data with the scores
with open(path, "w") as f:
    json.dump(data, f, indent=4)